In [2]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split


#### Cargo los Datos

In [3]:
data=pd.read_csv("../data/data.csv")
data.head()

,clasificacion,texto
0,1,El presidente abre la puerta a unos comicios e...
1,1,"En un escrito, la abogada del Estado Rosa Marí..."
2,0,Este lunes y martes la Asamblea de Madrid acog...
3,1,Resulta evidente que la ley ha tenido algunos ...
4,1,"El hispanista, que ya ocupó un puesto simbólic..."


In [4]:
data.value_counts("clasificacion")

clasificacion
1    70908
0    51868
Name: count, dtype: int64

In [5]:
data['texto']= data['texto'].astype(str)

In [6]:
data_chequeado=pd.read_csv("../data/chequeado.csv")
data_chequeado.head()

,clasificacion,texto
0,1,No se divulgaron resultados oficiales previos ...
1,0,Se difundieron en redes sociales supuestos pri...
2,1,"En el balotaje 2023, las boletas con el número..."
3,0,"En el balotaje 2023, las boletas con el número..."
4,0,"Miles de usuarios denuncian que, durante la jo..."


In [7]:
data_chequeado.value_counts("clasificacion")

clasificacion
0    1770
1     857
Name: count, dtype: int64

In [8]:
data_chequeado['texto']= data_chequeado['texto'].astype(str)

#### Divido en Train, Test Y Split

In [9]:
# Divido el conjunto de datos en entrenamiento (60%), prueba (20%) y validación (20%)
train_data, test_valid_data = train_test_split(data, test_size=0.4, random_state=42)
test_data, val_data = train_test_split(test_valid_data, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de prueba:", len(test_data))
print("Tamaño del conjunto de validación:", len(val_data))

Tamaño del conjunto de entrenamiento: 73665
Tamaño del conjunto de prueba: 24555
Tamaño del conjunto de validación: 24556


In [10]:
# Divido el conjunto de datos en entrenamiento (60%), prueba (20%) y validación (20%)
train_chequeado, test_valid_chequeado= train_test_split(data_chequeado, test_size=0.4, random_state=42)
test_chequeado, val_chequeado = train_test_split(test_valid_chequeado, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_chequeado))
print("Tamaño del conjunto de prueba:", len(test_chequeado))
print("Tamaño del conjunto de validación:", len(val_chequeado))

Tamaño del conjunto de entrenamiento: 1576
Tamaño del conjunto de prueba: 525
Tamaño del conjunto de validación: 526


#### Transformo los Datos con Spacy

In [28]:
def convert_pandas_to_tuple(data):
    data = data.to_dict(orient='records')
    return [(d["texto"], {"FALSE": d["clasificacion"] == 0, "TRUE": d["clasificacion"] == 1}) for d in data]

In [33]:
print(tuple_train_data[1])

('Ábalos deja claro que “un Gobierno de colaboración no es un Gobierno de coalición”, pero dice que la decisión de que haya ministros de Podemos le corresponde al presidente.', {'FALSE': False, 'TRUE': True})


In [29]:
tuple_train_data = convert_pandas_to_tuple(train_data)
tuple_val_data = convert_pandas_to_tuple(val_data)
tuple_test_data = convert_pandas_to_tuple(test_data)

In [30]:
tuple_train_chequeado = convert_pandas_to_tuple(train_chequeado)
tuple_val_chequeado = convert_pandas_to_tuple(val_chequeado)
tuple_test_chequeado = convert_pandas_to_tuple(test_chequeado)

In [ ]:
nlp = spacy.load("es_core_news_lg")

In [31]:
def process_docs(data,nlp):
    docs = []
    for doc, context in nlp.pipe(data, as_tuples=True):
        doc.cats = context
        docs.append(doc)
    return docs

In [ ]:
doc_train_data = process_docs(tuple_train_data,nlp)

In [71]:
doc_test_data = process_docs(tuple_test_data,nlp)

In [72]:
doc_val_data = process_docs(tuple_val_data,nlp)

In [75]:
doc_train_chequeado = process_docs(tuple_train_chequeado,nlp)

In [76]:
doc_test_chequeado = process_docs(tuple_test_chequeado,nlp)

In [77]:
doc_val_chequeado = process_docs(tuple_val_chequeado,nlp)

#### Almaceno los Docs

In [66]:
from spacy.tokens import DocBin

In [67]:
def save_docs(data,path):
    docbin = DocBin(docs=data)
    docbin.to_disk(path)

In [70]:
save_docs(doc_train_data,"./docs/train_data.spacy")

In [73]:
save_docs(doc_test_data,"./docs/test_data.spacy")

In [74]:
save_docs(doc_val_data,"./docs/val_data.spacy")

In [78]:
save_docs(doc_train_chequeado,"./docs/train_chequeado.spacy")

In [79]:
save_docs(doc_test_chequeado,"./docs/test_chequeado.spacy")

In [80]:
save_docs(doc_val_chequeado,"./docs/val_chequeado.spacy")

: 